In [1]:
import os

In [2]:
%pwd

'c:\\Project\\Insurance-ml-Project\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'c:\\Project\\Insurance-ml-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:

from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from src.mlProject.logging import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import joblib
import os

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import joblib
from pathlib import Path

class DataTransformationConfig:
    def __init__(self, root_dir: str, data_path: str, categorical_features: list):
        self.root_dir = root_dir
        self.data_path = data_path
        self.categorical_features = categorical_features

class DataTransformation:
    def __init__(self, config):
        self.config = config
        
    def train_test_spliting(self, categorical_features):
        data_path = self.config.data_path
        
        # Load the data
        data = pd.read_csv(data_path)

        # Separate features and target variable
        X = data.drop(columns=['expenses'])
        y = data['expenses']

        # One-hot encode categorical features
        encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_features]))
        X_encoded.columns = encoder.get_feature_names_out(input_features=categorical_features)
        X.drop(columns=categorical_features, inplace=True)
        X = pd.concat([X, X_encoded], axis=1)

        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Add 'expenses' column back to train and test data
        X_train['expenses'] = y_train
        X_test['expenses'] = y_test

        # Create the directory if it doesn't exist
        data_transformation_dir = Path(self.config.root_dir) / 'artifacts' / 'data_transformation'
        data_transformation_dir.mkdir(parents=True, exist_ok=True)

        # Save the encoder inside data_transformation directory
        encoder_path = data_transformation_dir / 'encoder.pkl'
        joblib.dump(encoder, encoder_path)

        # Save the transformed training and test data inside data_transformation directory
        train_csv_path = data_transformation_dir / 'train.csv'
        test_csv_path = data_transformation_dir / 'test.csv'
        X_train.to_csv(train_csv_path, index=False)
        X_test.to_csv(test_csv_path, index=False)

CONFIG_FILE_PATH = "full/path/to/config.yaml"
PARAMS_FILE_PATH = "full/path/to/params.yaml"
SCHEMA_FILE_PATH = "full/path/to/schema.yaml"

# Usage:
try:
    config = ConfigurationManager()
    data_transformation_config = DataTransformationConfig(root_dir='c:\\Project\\Insurance-ml-Project', data_path='research/insurance.csv', categorical_features=['sex', 'smoker', 'region'])
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting(data_transformation_config.categorical_features)
except Exception as e:
    raise e


[2024-03-01 03:13:12,417: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-01 03:13:12,431: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-01 03:13:12,440: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-01 03:13:12,440: INFO: common: created directory at: artifacts]


c:\Project\Insurance-ml-Project\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
